# 助教请看全局配置处的配置，现在配置的是仅推理：
```python
class Config:
    # --- 核心开关 (助教请注意这里) ---
    ONLY_INFERENCE = True  # 如果只想生成 submission.csv 而不进行训练，请将此设置为 True
```
推理产生submission.csv可能需要10分钟左右
# 参数量的打印代码位于推理产生submission的代码前20行的位置
# 如果需要查看参数量的打印内容，烦请稍等2分钟，从huggingface上下载我微调完成的模型权重；
# 即将开始推理时，参数量会彩色高亮打印出来

In [1]:
# !pip install datasets rouge_score emoji

这个error不用管，会打印5次之后正常运行：
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
import pandas as pd
import os
import re
import numpy as np
import csv
import time
import math
import emoji
import matplotlib.pyplot as plt
import seaborn as sns
from contextlib import nullcontext
from collections import OrderedDict
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BartForConditionalGeneration, 
    BartConfig, 
    AutoTokenizer,
    get_scheduler
)
from huggingface_hub import snapshot_download

2025-12-07 13:30:23.870921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765114224.043288      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765114224.091714      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# 1. 全局配置 (Configuration)

In [3]:
# =============================================================================
# 1. 全局配置 (Configuration)
# =============================================================================

class Config:
    # --- 核心开关 (助教请注意这里) ---
    ONLY_INFERENCE = True  # 如果只想生成 submission.csv 而不进行训练，请将此设置为 True
    
    
    input_ta_train = '/kaggle/input/nanogpt-fudannlp-cs-30040/train.csv'   
    input_test = '/kaggle/input/nanogpt-fudannlp-cs-30040/test.csv'        

    # --- 工作目录 ---
    dataset_path = '/kaggle/working/data'
    train_clean_csv = 'train_clean.csv'       
    val_clean_csv = 'validation_clean.csv'    

    # --- HuggingFace 权重配置  ---
    hf_repo_id = 'Kevin36277/finetuned-model'
    hf_subfolder = 'out-new-v1' 
    download_dir = '/kaggle/working/downloaded_model'

    # --- 模型配置 ---
    base_model_name = 'facebook/bart-large'
    
    # 训练输出路径
    out_dir = '/kaggle/working/out-v1-optimized'
    
    # --- 训练超参数 ---
    batch_size = 8#64
    gradient_accumulation_steps = 1
    max_iters = 2#3600
    
    # 长度控制
    max_dialogue_tokens = 364  
    max_summary_tokens = 64    
    max_source_length = 384
    max_target_length = 64
    min_length = 11
    
    # 优化器
    learning_rate = 5e-5
    weight_decay = 0.01
    beta1 = 0.9
    beta2 = 0.999
    grad_clip = 1.0
    label_smoothing = 0.0
    
    # 正则化 & 剪枝
    use_rdrop = True
    rdrop_alpha = 0.7
    dropout_rate = 0.07
    attention_dropout = 0.1
    activation_dropout = 0.1
    decoder_layers = 11
    
    # 生成配置
    num_beams = 8
    length_penalty = 1.0
    no_repeat_ngram_size = 3
    
    # 调度器
    lr_scheduler_type = "cosine"
    warmup_iters = 200
    
    # 早停
    use_early_stopping = True
    early_stopping_patience = 5
    
    # I/O
    eval_interval = 200
    log_interval = 50
    eval_iters = 40
    eval_rouge_during_training = True
    rouge_eval_samples = 500
    
    # 系统
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype = 'float16' 
    compile = False#不是A100，不支持True
    use_multi_gpu = True
    num_workers = 4 
    persistent_workers = True

config = Config()

# 精度回退检查
if torch.cuda.is_available():
    if not (torch.cuda.is_bf16_supported() and config.dtype == 'bfloat16'):
        print("⚠️  GPU 不支持 bfloat16 或未启用，回退到 float16")
        config.dtype = 'float16'
else:
    config.device = 'cpu'
    config.dtype = 'float32'

⚠️  GPU 不支持 bfloat16 或未启用，回退到 float16


# 2. 工具函数：下载与加载

In [4]:
# =============================================================================
# 2. 工具函数：下载与加载
# =============================================================================

def download_model_from_hf():
    """从 HuggingFace 下载指定文件夹下的所有模型文件"""
    print(f"\n>>> 正在从 HuggingFace 下载模型权重...")
    print(f"    Repo: {config.hf_repo_id}")
    print(f"    Folder: {config.hf_subfolder}")
    
    try:
        local_path = snapshot_download(
            repo_id=config.hf_repo_id,
            allow_patterns=f"{config.hf_subfolder}/*",
            local_dir=config.download_dir,
            local_dir_use_symlinks=False 
        )
        
        model_path = os.path.join(local_path, config.hf_subfolder)
        print(f"    ✅ 下载完成。模型位于: {model_path}")
        return model_path
    except Exception as e:
        print(f"    ❌ 下载失败: {e}")
        return None

# 3. 数据处理 (Preprocessing)

In [5]:
# =============================================================================
# 3. 数据处理 (Preprocessing)
# =============================================================================

def clean_text_remove_emoji(text):
    """使用 emoji 库精准删除表情"""
    if not isinstance(text, str): 
        return ""
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def process_dataframe(df, tokenizer, name_tag, is_test=False):
    """处理DataFrame，统一列名、清洗、筛选"""
    if df is None or len(df) == 0: 
        return pd.DataFrame()
    print(f"    >>> 处理: {name_tag} (原始: {len(df)})")
    
    df = df.copy()
    # 统一列名
    col_map = {}
    for col in df.columns:
        if col.lower() in ['text', 'document', 'dialogue', 'content']: 
            col_map[col] = 'dialogue'
        elif col.lower() in ['summary', 'target', 'headline']: 
            col_map[col] = 'summary'
    if col_map: 
        df.rename(columns=col_map, inplace=True)

    if 'id' not in df.columns: 
        df['id'] = [f"{name_tag}_{i}" for i in range(len(df))]
    
    df.replace("", pd.NA, inplace=True)
    subset = ['dialogue'] if is_test else ['dialogue', 'summary']
    df.dropna(subset=subset, inplace=True)
    
    # 去除表情
    df['dialogue'] = df['dialogue'].apply(clean_text_remove_emoji)
    if not is_test: 
        df['summary'] = df['summary'].apply(clean_text_remove_emoji)
    
    # 长度筛选 (仅训练)
    if not is_test:
        df['d_len'] = df['dialogue'].apply(lambda x: len(tokenizer.encode(str(x), add_special_tokens=False)))
        df['s_len'] = df['summary'].apply(lambda x: len(tokenizer.encode(str(x), add_special_tokens=False)))
        before_len = len(df)
        df = df[
            (df['d_len'] <= config.max_dialogue_tokens) & 
            (df['s_len'] <= config.max_summary_tokens) &
            (df['d_len'] > 10) & 
            (df['s_len'] > 10)
        ]
        print(f"    [长度筛选] 移除了 {before_len - len(df)} 行")
    
    return df

def prepare_data_pipeline():
    """数据准备流水线"""
    print("=" * 60)
    print("STEP 1: 数据准备 (Data Prep)")
    print("=" * 60)
    os.makedirs(config.dataset_path, exist_ok=True)
    
    # 尝试加载 tokenizer 用于长度计算
    try:
        tokenizer = AutoTokenizer.from_pretrained(config.base_model_name)
    except:
        tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')

    # 1. TA 数据
    if os.path.exists(config.input_ta_train):
        df_ta = pd.read_csv(config.input_ta_train)
        df_ta = process_dataframe(df_ta, tokenizer, "TA_Train")
        
        # 简单划分
        df_ta = df_ta.sample(frac=1, random_state=42).reset_index(drop=True)
        val_size = 700 if len(df_ta) > 2000 else int(len(df_ta) * 0.1)
        df_val = df_ta.iloc[-val_size:].copy()
        df_train = df_ta.iloc[:-val_size].copy()
        
        # 保存时去除长度列
        train_path = os.path.join(config.dataset_path, config.train_clean_csv)
        val_path = os.path.join(config.dataset_path, config.val_clean_csv)
        save_cols = ['id', 'dialogue', 'summary']
        df_train[save_cols].to_csv(train_path, index=False)
        df_val[save_cols].to_csv(val_path, index=False)
        print(f"    ✅ 训练集: {len(df_train)} 条")
        print(f"    ✅ 验证集: {len(df_val)} 条")
        print(f"    ✅ 数据已保存至 {config.dataset_path}")
    else:
        print(f"    ⚠️ 未找到训练数据 {config.input_ta_train}，跳过数据准备(如果是推理模式可忽略)")

# 4. Dataset 类

In [6]:
# =============================================================================
# 4. Dataset 类
# =============================================================================

class SummarizationDataset(Dataset):
    """优化版数据集类"""
    def __init__(self, csv_path, tokenizer, max_source_length, max_target_length, mode='train'):
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        self.mode = mode
        
        self.dialogues = []
        self.summaries = []
        self.ids = []
        
        if os.path.exists(csv_path):
            with open(csv_path, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    self.dialogues.append(row['dialogue'])
                    if 'id' in row:
                        self.ids.append(row['id'])
                    if mode != 'test' and 'summary' in row:
                        self.summaries.append(row['summary'])
            print(f"  加载了 {len(self.dialogues)} 个样本 from {csv_path}")
        else:
            raise FileNotFoundError(f"找不到文件: {csv_path}")
    
    def __len__(self):
        return len(self.dialogues)
    
    def __getitem__(self, idx):
        dialogue = str(self.dialogues[idx])
        
        source = self.tokenizer(
            dialogue,
            max_length=self.max_source_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        item = {
            'input_ids': source['input_ids'].squeeze(),
            'attention_mask': source['attention_mask'].squeeze(),
        }
        
        if self.mode != 'test' and idx < len(self.summaries):
            summary = str(self.summaries[idx])
            target = self.tokenizer(
                text_target=summary,
                max_length=self.max_target_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            labels = target['input_ids'].squeeze()
            labels[labels == self.tokenizer.pad_token_id] = -100
            item['labels'] = labels
        
        if idx < len(self.ids):
            item['id'] = self.ids[idx]
        
        return item

# 5. 辅助函数

In [7]:
# =============================================================================
# 5. 辅助函数
# =============================================================================

def get_grouped_parameters(model, weight_decay):
    """
    优化器参数分组：Bias 和 LayerNorm 不使用 weight decay
    这是 BERT/BART 训练的标准做法
    """
    no_decay = ["bias", "LayerNorm.weight", "layer_norm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() 
                      if not any(nd in n for nd in no_decay) and p.requires_grad],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() 
                      if any(nd in n for nd in no_decay) and p.requires_grad],
            "weight_decay": 0.0,
        },
    ]
    return optimizer_grouped_parameters

def compute_kl_loss(logits1, logits2):
    """
    优化版 KL Loss：使用 reduction='batchmean'
    """
    vocab_size = logits1.size(-1)
    logits1_flat = logits1.view(-1, vocab_size)
    logits2_flat = logits2.view(-1, vocab_size)
    
    # 双向 KL 散度
    kl_loss = F.kl_div(
        F.log_softmax(logits1_flat, dim=-1),
        F.softmax(logits2_flat, dim=-1),
        reduction='batchmean'
    ) + F.kl_div(
        F.log_softmax(logits2_flat, dim=-1),
        F.softmax(logits1_flat, dim=-1),
        reduction='batchmean'
    )
    
    return kl_loss / 2

@torch.no_grad()
def estimate_loss(model, dataloader_dict, ctx):
    """估算验证集损失"""
    out = {}
    model.eval()
    
    for split in ['train', 'val']:
        losses = []
        dataloader = dataloader_dict[split]
        
        for i, batch in enumerate(dataloader):
            if i >= config.eval_iters:
                break
            
            # 过滤掉非tensor的键（如'id'）
            batch = {k: v.to(config.device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
            
            with ctx:
                outputs = model(**batch)
                loss = outputs.loss
                if hasattr(model, 'module'):
                    loss = loss.mean()
            
            losses.append(loss.item())
        
        out[split] = np.mean(losses) if losses else float('inf')
    
    model.train()
    return out

def calculate_rouge(reference_summary, generated_summary):
    """计算 ROUGE 分数"""
    try:
        from rouge_score import rouge_scorer
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(reference_summary, generated_summary)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }
    except ImportError:
        return None

@torch.no_grad()
def evaluate_rouge(model, tokenizer, val_dataloader, ctx, num_samples=10):
    """评估 ROUGE"""
    eval_model = model.module if hasattr(model, 'module') else model
    eval_model.eval()
    
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    
    print(f"  正在计算 ROUGE (采样 {num_samples} 条)...")
    print_count = 0
    samples_processed = 0
    
    for batch in val_dataloader:
        if samples_processed >= num_samples:
            break
        
        input_ids = batch['input_ids'].to(config.device)
        attention_mask = batch['attention_mask'].to(config.device)
        labels = batch['labels']
        
        # 解码参考摘要
        labels_copy = labels.clone()
        labels_copy[labels_copy == -100] = tokenizer.pad_token_id
        ref_summaries = tokenizer.batch_decode(labels_copy, skip_special_tokens=True)
        
        with ctx:
            outputs = eval_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=config.max_target_length,
                min_length=config.min_length,
                num_beams=config.num_beams,
                length_penalty=config.length_penalty,
                no_repeat_ngram_size=config.no_repeat_ngram_size,
                early_stopping=True
            )
        
        pred_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        for ref, pred in zip(ref_summaries, pred_summaries):
            if print_count < 3:
                print(f"\n[Case {print_count + 1}]")
                print(f"Ref : {ref}")
                print(f"Pred: {pred}")
                print("-" * 30)
                print_count += 1
            
            rouge_scores = calculate_rouge(ref, pred)
            if rouge_scores:
                rouge1_scores.append(rouge_scores['rouge1'])
                rouge2_scores.append(rouge_scores['rouge2'])
                rougeL_scores.append(rouge_scores['rougeL'])
            
            samples_processed += 1
            if samples_processed >= num_samples:
                break
    
    eval_model.train()
    
    if len(rouge1_scores) > 0:
        return {
            'rouge1': np.mean(rouge1_scores),
            'rouge2': np.mean(rouge2_scores),
            'rougeL': np.mean(rougeL_scores)
        }
    return None

# 6. 训练主流程 (Train Loop)

In [8]:
# =============================================================================
# 6. 训练主流程 (Train Loop)
# =============================================================================

def train(model_load_path):
    """训练主流程（集成所有高级策略）"""
    print("=" * 60)
    print("STEP 2: 模型训练 (Training)")
    print("=" * 60)
    
    torch.manual_seed(42)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    os.makedirs(config.out_dir, exist_ok=True)
    
    device_type = 'cuda' if 'cuda' in config.device else 'cpu'
    ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[config.dtype]
    ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
    
    print(f"使用精度: {config.dtype} ({ptdtype})")
    
    # --- 加载 Tokenizer ---
    print(f"\n加载 Tokenizer from: {model_load_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_load_path)
    
    # --- 构建 DataLoader ---
    print("\n构建 DataLoader...")
    train_path = os.path.join(config.dataset_path, config.train_clean_csv)
    val_path = os.path.join(config.dataset_path, config.val_clean_csv)
    
    train_dataset = SummarizationDataset(train_path, tokenizer, config.max_source_length, config.max_target_length, mode='train')
    val_dataset = SummarizationDataset(val_path, tokenizer, config.max_source_length, config.max_target_length, mode='val')
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=True,
        persistent_workers=config.persistent_workers if config.num_workers > 0 else False,
        drop_last=True if config.use_rdrop else False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=True,
        persistent_workers=config.persistent_workers if config.num_workers > 0 else False
    )
    
    dataloader_dict = {'train': train_loader, 'val': val_loader}
    
    # --- 模型初始化 ---
    print(f"\n加载 Model from: {model_load_path}")
    print(f"配置模型 (Dropout={config.dropout_rate}, Att={config.attention_dropout}, Act={config.activation_dropout})...")
    
    # 配置模型
    model_config = BartConfig.from_pretrained(model_load_path)
    model_config.dropout = config.dropout_rate
    model_config.attention_dropout = config.attention_dropout
    model_config.activation_dropout = config.activation_dropout
    model_config.classif_dropout = 0.0
    model_config.decoder_layers = config.decoder_layers
    
    model = BartForConditionalGeneration.from_pretrained(model_load_path, config=model_config)
    
    # 剪枝decoder层
    if len(model.model.decoder.layers) > config.decoder_layers:
        print(f"执行模型剪枝: {len(model.model.decoder.layers)} -> {config.decoder_layers} 层")
        model.model.decoder.layers = model.model.decoder.layers[:config.decoder_layers]
    
    num_params = sum(p.numel() for p in model.parameters())
    print(f"\n>>> 模型参数量: {num_params / 1e6:.2f} M <<<")
    
    model.to(config.device)
    
    # 保存原始模型引用（用于保存checkpoint）
    raw_model = model
    
    # 多GPU（必须在 torch.compile 之前）
    if torch.cuda.device_count() > 1 and config.use_multi_gpu:
        print(f"\n使用 {torch.cuda.device_count()} 个GPU训练")
        model = nn.DataParallel(model)
        raw_model = model.module
        # 多GPU时禁用compile，因为二者不兼容
        config.compile = False
    
    # torch.compile 优化（仅单GPU时使用）
    if config.compile and hasattr(torch, 'compile'):
        print("\n正在编译模型 (torch.compile)...")
        try:
            model = torch.compile(model)
            print("✅ 模型编译成功！预期加速 15-30%")
        except Exception as e:
            print(f"⚠️  编译失败（将继续使用非编译版本）: {e}")
    
    # --- 优化器参数分组 ---
    print("\n构建优化器（参数分组）...")
    optimizer_grouped_parameters = get_grouped_parameters(raw_model, config.weight_decay)
    optimizer = torch.optim.AdamW(
        optimizer_grouped_parameters,
        lr=config.learning_rate,
        betas=(config.beta1, config.beta2)
    )
    
    # 使用 transformers 原生调度器
    print(f"构建学习率调度器 ({config.lr_scheduler_type})...")
    lr_scheduler = get_scheduler(
        name=config.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=config.warmup_iters,
        num_training_steps=config.max_iters
    )
    
    # Scaler（修复废弃警告）
    scaler = torch.amp.GradScaler('cuda', enabled=(config.dtype == 'float16'))
    
    # --- 训练循环 ---
    print("\n开始训练...")
    iter_num = 0
    best_val_loss = 1e9
    best_rouge_sum = 0.0
    patience_counter = 0
    t0 = time.time()
    
    progress_bar = tqdm(total=config.max_iters, desc="Training", ncols=120)
    
    # 无限迭代器
    def infinite_loader(loader):
        while True:
            for batch in loader:
                yield batch
    
    train_iter = infinite_loader(train_loader)
    
    while iter_num < config.max_iters:
        # --- 评估 ---
        if iter_num > 0 and iter_num % config.eval_interval == 0:
            losses = estimate_loss(model, dataloader_dict, ctx)
            print(f"\n[Step {iter_num}] train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            
            rouge_scores = None
            current_rouge_sum = 0.0
            
            if config.eval_rouge_during_training:
                rouge_scores = evaluate_rouge(model, tokenizer, val_loader, ctx, num_samples=config.rouge_eval_samples)
                if rouge_scores:
                    current_rouge_sum = rouge_scores['rouge1'] + rouge_scores['rouge2'] + rouge_scores['rougeL']
                    print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f} | ROUGE-2: {rouge_scores['rouge2']:.4f} | ROUGE-L: {rouge_scores['rougeL']:.4f}")
                    print(f"  平均分: {current_rouge_sum / 3:.4f}")
            
            # 保存策略
            saved = False
            if rouge_scores and current_rouge_sum > best_rouge_sum:
                best_rouge_sum = current_rouge_sum
                patience_counter = 0
                print(f"  >>> 新的最佳模型 (ROUGE Sum: {current_rouge_sum:.4f})! 保存中... <<<")
                saved = True
            elif losses['val'] < best_val_loss and not rouge_scores:
                best_val_loss = losses['val']
                patience_counter = 0
                print(f"  新的最佳验证损失: {best_val_loss:.4f}。保存checkpoint。")
                saved = True
            else:
                patience_counter += 1
                print(f"  耐心计数: {patience_counter}/{config.early_stopping_patience}")
            
            if saved:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'lr_scheduler': lr_scheduler.state_dict(),
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'best_rouge_sum': best_rouge_sum,
                    'config': vars(config),
                }
                torch.save(checkpoint, os.path.join(config.out_dir, 'ckpt.pt'))
                raw_model.save_pretrained(config.out_dir)
                tokenizer.save_pretrained(config.out_dir)
            
            if config.use_early_stopping and patience_counter >= config.early_stopping_patience:
                print(f"早停触发。")
                break
            
            print("-" * 50)
        
        # --- Forward + Backward ---
        model.train()
        
        batch = next(train_iter)
        # 过滤掉非tensor的键（如'id'）
        batch = {k: v.to(config.device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        
        with ctx:
            if config.use_rdrop:
                # R-Drop: 两次前向传播
                outputs1 = model(**batch)
                outputs2 = model(**batch)
                
                logits1 = outputs1.logits
                logits2 = outputs2.logits
                
                # 交叉熵损失（使用模型自带的loss）
                ce_loss = (outputs1.loss + outputs2.loss) / 2
                
                # KL散度损失
                kl_loss = compute_kl_loss(logits1, logits2)
                
                loss = ce_loss + config.rdrop_alpha * kl_loss
            else:
                outputs = model(**batch)
                loss = outputs.loss
            
            # 多GPU时需要mean
            if hasattr(model, 'module'):
                loss = loss.mean()
            
            loss = loss / config.gradient_accumulation_steps
        
        # Backward
        if config.dtype == 'bfloat16':
            loss.backward()
        else:
            scaler.scale(loss).backward()
        
        # 梯度累积与更新
        if (iter_num + 1) % config.gradient_accumulation_steps == 0:
            if config.dtype == 'bfloat16':
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
                optimizer.step()
            else:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
                scaler.step(optimizer)
                scaler.update()
            
            lr_scheduler.step()
            optimizer.zero_grad(set_to_none=True)
        
        # 日志
        if iter_num % config.log_interval == 0:
            t1 = time.time()
            lossf = loss.item() * config.gradient_accumulation_steps
            current_lr = lr_scheduler.get_last_lr()[0]
            dt = (t1 - t0) * 1000 / max(config.log_interval, 1)
            progress_bar.set_postfix(loss=f"{lossf:.4f}", lr=f"{current_lr:.2e}", ms=f"{dt:.2f}")
            t0 = t1
        
        iter_num += 1
        progress_bar.update(1)
    
    # 训练结束，保存最终模型
    checkpoint = {
        'model': raw_model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'lr_scheduler': lr_scheduler.state_dict(),
        'iter_num': iter_num,
        'best_val_loss': best_val_loss,
        'best_rouge_sum': best_rouge_sum,
        'config': vars(config),
    }
    torch.save(checkpoint, os.path.join(config.out_dir, 'ckpt.pt'))
    raw_model.save_pretrained(config.out_dir)
    tokenizer.save_pretrained(config.out_dir)
    
    progress_bar.close()
    print("\n训练完成！最终模型已保存。")
    return config.out_dir

# 7. 推理与提交 (Generate Submission)

In [9]:
# =============================================================================
# 7. 推理与提交 (Generate Submission)
# =============================================================================

def generate_submission(model_path):
    """生成测试集提交文件"""
    print("=" * 60)
    print("STEP 3: 生成提交文件 (Inference)")
    print("=" * 60)
    
    if not os.path.exists(config.input_test):
        print("❌ 未找到测试集，无法生成提交。")
        return

    print(f"加载模型进行推理: {model_path}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = BartForConditionalGeneration.from_pretrained(model_path)
        n_params = sum(p.numel() for p in model.parameters())
        param_display = f"{n_params/1e6:.2f} M"
        
        
        n_params = sum(p.numel() for p in model.parameters())
        val_m = n_params/1e6
        
        # 定义颜色代码
        C_BORDER = "\033[1;36m"  # 亮青色边框
        C_TITLE  = "\033[1;33m"  # 亮黄色标题
        C_PASS   = "\033[1;32m"  # 亮绿色通过
        C_FAIL   = "\033[1;31m"  # 亮红色失败
        C_RESET  = "\033[0m"     # 重置
        
        # 构造信息字符串
        if val_m < 400:
            status_line = f"{C_PASS}PASS: {val_m:.2f} M < 400 M{C_RESET}"
            
            padding_len = 52 - len(f"PASS: {val_m:.2f} M < 400 M")
        else:
            status_line = f"{C_FAIL}FAIL: {val_m:.2f} M > 400 M{C_RESET}"
            padding_len = 52 - len(f"FAIL: {val_m:.2f} M > 400 M")
        
        padding = " " * max(0, padding_len)

        # 打印全彩 ASCII 框
        print("\n" * 3)  # 制造垂直隔离区
        print(f"{C_BORDER}{'='*60}{C_RESET}")
        print(f"{C_BORDER}!!{C_RESET}{' '*56}{C_BORDER}!!{C_RESET}")
        print(f"{C_BORDER}!!{C_RESET}   {C_TITLE}>>> 助教请看这里 (TA CHECKPOINT) <<<{C_RESET}   {C_BORDER}!!{C_RESET}")
        print(f"{C_BORDER}!!{C_RESET}{' '*56}{C_BORDER}!!{C_RESET}")
        print(f"{C_BORDER}!!{C_RESET}   {status_line}{padding}{C_BORDER}!!{C_RESET}")
        print(f"{C_BORDER}!!{C_RESET}{' '*56}{C_BORDER}!!{C_RESET}")
        print(f"{C_BORDER}{'='*60}{C_RESET}")
        print("\n" * 2) 

        # 原始要求代码
        assert n_params/1e6 < 400, f"参数量超标: {val_m:.2f}M > 400M"
        
    except Exception as e:
        print(f"❌ 加载模型失败: {e}")
        return

    model.to(config.device)
    model.eval()
    
    test_df = pd.read_csv(config.input_test)
    # 列名适配
    col_map = {}
    for col in test_df.columns:
        if col.lower() in ['text', 'document', 'content']: 
            col_map[col] = 'dialogue'
    if col_map: 
        test_df.rename(columns=col_map, inplace=True)
    
    test_df['dialogue'] = test_df['dialogue'].apply(clean_text_remove_emoji).fillna("")
    print(f"推理数据量: {len(test_df)}")
    print("下面的进度条不是x/2273，这是因为我是批量推理的，/右边的是批次数量")
    results = []
    batch_size = 64
    dialogues = test_df['dialogue'].tolist()
    
    device_type = 'cuda' if 'cuda' in config.device else 'cpu'
    ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[config.dtype]
    ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
    
    with torch.no_grad():
        for i in tqdm(range(0, len(dialogues), batch_size), desc="Generating"):
            batch = dialogues[i:i+batch_size]
            inputs = tokenizer(
                batch, 
                max_length=config.max_source_length, 
                padding='max_length', 
                truncation=True, 
                return_tensors='pt'
            ).to(config.device)
            
            with ctx:
                outputs = model.generate(
                    inputs['input_ids'], 
                    attention_mask=inputs['attention_mask'],
                    num_beams=config.num_beams, 
                    max_length=config.max_target_length, 
                    min_length=config.min_length,
                    length_penalty=config.length_penalty, 
                    no_repeat_ngram_size=config.no_repeat_ngram_size, 
                    early_stopping=True
                )
            
            decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            results.extend([s.strip() for s in decoded])
            
    submission_path = '/kaggle/working/submission.csv'
    pd.DataFrame({'id': test_df['id'], 'summary': results}).to_csv(submission_path, index=False)
    print(f"✅ Submission 已保存: {submission_path}")
    print("预览前5行：")
    print(pd.DataFrame({'id': test_df['id'][:5], 'summary': results[:5]}))

In [10]:
import random

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # 保证 CUDA 卷积操作的确定性 (会稍微牺牲一点点速度，但保证结果一致)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🔒 全局随机种子已固定为: {seed}")

# 8. 主程序入口

In [ ]:
# =============================================================================
# 8. 主程序入口
# =============================================================================

if __name__ == '__main__':
    # 1. 尝试从 HF 下载已微调的权重
    print("从 HF 下载已微调的权重，预计30秒")
    hf_model_path = download_model_from_hf()
    
    # 如果下载失败，回退到原始模型
    if hf_model_path is None:
        print("⚠️ 无法从 HF 下载，将使用 facebook/bart-large 原始权重")
        current_model_path = config.base_model_name
    else:
        current_model_path = hf_model_path
        
    seed_everything()
    print("随机种子已固定")
    # 2. 根据开关决定是 仅推理 还是 训练+推理
    if config.ONLY_INFERENCE:
        print("\n>>> 模式: 仅推理 (ONLY_INFERENCE = True)")
        generate_submission(current_model_path)
    
    else:
        print("\n>>> 模式: 训练 + 推理 (ONLY_INFERENCE = False)")
        # 准备数据
        prepare_data_pipeline()
        
        # 训练
        trained_model_path = train(current_model_path)
        
        # 使用刚训练好的权重进行推理
        generate_submission(trained_model_path)

从 HF 下载已微调的权重，预计30秒

>>> 正在从 HuggingFace 下载模型权重...
    Repo: Kevin36277/finetuned-model
    Folder: out-new-v1


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

merges.txt: 0.00B [00:00, ?B/s]

submission.csv-checkpoint.txt: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

    ✅ 下载完成。模型位于: /kaggle/working/downloaded_model/out-new-v1
🔒 全局随机种子已固定为: 42
随机种子已固定

>>> 模式: 仅推理 (ONLY_INFERENCE = True)
STEP 3: 生成提交文件 (Inference)
加载模型进行推理: /kaggle/working/downloaded_model/out-new-v1




!!                                                        !!
!!   >>> 助教请看这里 (TA CHECKPOINT) <<<   !!
!!                                                        !!
!!   PASS: 389.49 M < 400 M                              !!
!!                                                        !!



推理数据量: 2273
下面的进度条不是x/2273，这是因为我是批量推理的，/右边的是批次数量


Generating:  44%|████▍     | 16/36 [07:36<07:25, 22.27s/it]